# Preprocessing file format, #channel, and sample rate

In [83]:
# use pydub to convert all file to wav

from pydub import AudioSegment
AUDIO_FILE_PATH = 'test_pydub'
audio_file = 'test water mp3.mp3'
file_path = pathlib.Path(AUDIO_FILE_PATH+'/'+audio_file)

mp3_audio = AudioSegment.from_file(file_path, format="mp3")

In [ ]:
mp3_audio

In [54]:
print(f'{mp3_audio.frame_rate} {mp3_audio.channels} ')

44100 1 


In [60]:
mp3_audio_mono = mp3_audio.set_channels(2)
mp3_audio_mono_16k = mp3_audio_mono.set_frame_rate(16000)

In [61]:
output_file_name = 'output2.wav'
output = mp3_audio_mono_16k.export(pathlib.Path(AUDIO_FILE_PATH+'/'+output_file_name), 
                      format = "wav",
                     )
output.close()

In [62]:
#check the output
AUDIO_FILE_PATH = 'test_pydub'
audio_file = 'output2.wav'
file_path = pathlib.Path(AUDIO_FILE_PATH+'/'+audio_file)

wav_audio = AudioSegment.from_file(file_path, format="wav")

In [63]:
print(f'{wav_audio.frame_rate} {wav_audio.channels} ')

16000 2 


#### OK, so better to create separate function. one is for change everything to wav, and the other is to change the wav to correct channel and frame rate

In [115]:
# get all non-wav audio file in the folder

import os
import pathlib

# AUDIO_FILE_PATH = 'test_pydub'
AUDIO_FILE_PATH = 'test_1_home_recorded/test_pydub'

audio_file_dir = pathlib.Path(AUDIO_FILE_PATH)
dir_list = os.listdir(audio_file_dir)
dir_list

['non water sound sample.m4a',
 'non water sound sample.wav',
 'preprocessed_non water sound sample.wav',
 'preprocessed_water sound sample.wav',
 'test data water and non_water.m4a',
 'water sound sample.m4a',
 'water sound sample.wav']

In [116]:
non_wav_files = list(filter(lambda x: ".wav" not in x, dir_list))
non_wav_files

['non water sound sample.m4a',
 'test data water and non_water.m4a',
 'water sound sample.m4a']

In [117]:
for non_wav_file in non_wav_files:
    non_wav_audio = AudioSegment.from_file(pathlib.Path(AUDIO_FILE_PATH+'/'+non_wav_file), format=non_wav_file[-3:])
    output = non_wav_audio.export(pathlib.Path(AUDIO_FILE_PATH+'/'+non_wav_file[:-3]+'wav'), format = "wav")
    output.close()

# it works, all non wav now turned in to wav

In [122]:
# now, get all wav files, and change them to correct channel and frame rate
dir_list = os.listdir(audio_file_dir) # need to do listdir again
wav_files = list(filter(lambda x: (".wav" in x) and ("preprocessed_" not in x), dir_list))
wav_files

['non water sound sample.wav',
 'test data water and non_water.wav',
 'water sound sample.wav']

In [119]:
forile wav_f in wav_files:
    wav_audio = AudioSegment.from_file(pathlib.Path(AUDIO_FILE_PATH+'/'+wav_file), format="wav")
    wav_audio = wav_audio.set_channels(1)
    wav_audio = wav_audio.set_frame_rate(16000)
    output = wav_audio.export(pathlib.Path(AUDIO_FILE_PATH+'/preprocessed_'+wav_file), format = "wav")
    output.close()

In [120]:
# double check that everything is on right channels and frame rate
dir_list = os.listdir(audio_file_dir) # need to do listdir again
preprocessed_wav_files = list(filter(lambda x: "preprocessed_" in x, dir_list))
preprocessed_wav_files

['preprocessed_non water sound sample.wav',
 'preprocessed_preprocessed_non water sound sample.wav',
 'preprocessed_preprocessed_water sound sample.wav',
 'preprocessed_test data water and non_water.wav',
 'preprocessed_water sound sample.wav']

In [121]:
for wav_file in preprocessed_wav_files:
    wav_audio = AudioSegment.from_file(pathlib.Path(AUDIO_FILE_PATH+'/'+wav_file), format="wav")
    print(f'{wav_file} {wav_audio.channels} {wav_audio.frame_rate}')

preprocessed_non water sound sample.wav 1 16000
preprocessed_preprocessed_non water sound sample.wav 1 16000
preprocessed_preprocessed_water sound sample.wav 1 16000
preprocessed_test data water and non_water.wav 1 16000
preprocessed_water sound sample.wav 1 16000


In [ ]:
# it works, we now have converted all files to correct format #channel, and sample rate

# Preprocessing Sample Dataset

In [3]:
# get all timestamp csv file in the folder

import os
import pathlib

TIMESTAMP_PATH = 'timestamp'

timestamp_dir = pathlib.Path(TIMESTAMP_PATH)
dir_list = os.listdir(timestamp_dir)
dir_list

['time stamp 01.csv', 'time stamp 02.csv']

In [6]:
import pandas as pd

df = pd.read_csv(pathlib.Path(TIMESTAMP_PATH+'/'+dir_list[1]))
df

#OK this works to look at timestamp_path and dir

,file,start,stop,water
0,audio_2,0.0,0.7,0
1,audio_2,0.7,1.0,1
2,audio_2,1.0,1.5,0
3,audio_2,1.5,2.0,1


In [3]:
# let's make the function to split audio file to a new file
from pydub import AudioSegment

def split_audio(input_file_path, start_time, stop_time, output_file_path):
    t1 = start_time * 1000 #Works in milliseconds
    t2 = stop_time * 1000

    newAudio  = AudioSegment.from_wav(input_file_path)
    newAudio = newAudio[t1:t2]
    output = newAudio.export(output_file_path, format="wav")
    output.close()

In [30]:
import numpy as np
np.arange(0.7, 1.0, 0.5).tolist()

[0.7]

In [31]:
# try to split again the audio based on split length
import numpy as np
SPLIT_LENGTH = 0.5

for index, row in df.iterrows():
    start_split_time = np.arange(row['start'], row['stop'], SPLIT_LENGTH).tolist()
    stop_split_time = start_split_time[1:]+[row['stop']]
    print (f"Start: {row['start']} Stop: {row['stop']}")
    for start, stop in list(zip(start_split_time, stop_split_time)):
        print (f'{start} {stop}')
    
    
    
# print(start_split_time)
# print(stop_split_time)
# print(list(zip(start_split_time, stop_split_time)))

# OK it works

Start: 0.0 Stop: 0.7
0.0 0.5
0.5 0.7
Start: 0.7 Stop: 1.0
0.7 1.0
Start: 1.0 Stop: 1.5
1.0 1.5
Start: 1.5 Stop: 2.0
1.5 2.0


In [32]:
# ok the pandas read rows work, now try to iterate each and output the file in the folder
import pandas as pd
from pydub import AudioSegment

input_folder = r'test input folder'
output_folder = r'test result'


df = pd.read_csv(TIMESTAMP_PATH+'/'+dir_list[1])

for index, row in df.iterrows():
    start_split_time = np.arange(row['start'], row['stop'], SPLIT_LENGTH).tolist()
    stop_split_time = start_split_time[1:]+[row['stop']]
    print (f"Start: {row['start']} Stop: {row['stop']}")
    for start, stop in list(zip(start_split_time, stop_split_time)):
        print (f'{start} {stop}')    
        output_file_name = row['file']+'_'+str(start)+'_'+str(stop)
        if row['water'] == 1:
            split_audio(input_folder+'\\'+row['file']+'.wav', start, stop, output_folder+'\\water\\'+output_file_name+'.wav')
        else:
            split_audio(input_folder+'\\'+row['file']+'.wav', start, stop, output_folder+'\\non_water\\'+output_file_name+'.wav')
        
    
# this worksss! it split each audio file to water and non_water folder

Start: 0.0 Stop: 0.7
0.0 0.5
0.5 0.7
Start: 0.7 Stop: 1.0
0.7 1.0
Start: 1.0 Stop: 1.5
1.0 1.5
Start: 1.5 Stop: 2.0
1.5 2.0


In [114]:
# now expand the functionality it to reading each CSV files. This is for sample dataset
#need to manually create the folder , including water and nonwater folder

# ok the pandas read rows work, now try to iterate each and output the file in the folder
import pandas as pd
from pydub import AudioSegment
import os
import pathlib
import numpy as np

# TIMESTAMP_PATH = 'timestamp'
# sample_dataset_folder = r'test_sample_dataset'
# split_sample_dataset_folder = r'test_split_sample_dataset'

TIMESTAMP_PATH = 'test_1_home_recorded/timestamp'
sample_dataset_folder = r'test_1_home_recorded\test_sample_dataset'
split_sample_dataset_folder = r'test_1_home_recorded\test_split_sample_dataset'


SPLIT_LENGTH = 0.5


timestamp_dir = pathlib.Path(TIMESTAMP_PATH)
dir_list = os.listdir(timestamp_dir)

for csv_file in dir_list: 
    df = pd.read_csv(TIMESTAMP_PATH+'/'+csv_file)
    print(f"CSV file: {csv_file}")
    for index, row in df.iterrows():
        start_split_time = np.arange(row['start'], row['stop'], SPLIT_LENGTH).tolist()
        stop_split_time = start_split_time[1:]+[row['stop']]
        print (f"  audio file: {row['file']}  Start: {row['start']} Stop: {row['stop']}")
        if row['water'] == 1:
            print(f'    water')
        else:
            print(f'    non_water')
        for start, stop in list(zip(start_split_time, stop_split_time)):
            print (f'      {start} {stop}')
            output_file_name = row['file']+'_'+str(start)+'_'+str(stop)
            if row['water'] == 1:
                split_audio(sample_dataset_folder+'\\'+row['file']+'.wav', start, stop, split_sample_dataset_folder+'\\water\\'+output_file_name+'.wav')
            else:
                split_audio(sample_dataset_folder+'\\'+row['file']+'.wav', start, stop, split_sample_dataset_folder+'\\non_water\\'+output_file_name+'.wav')
    


CSV file: time stamp 01.csv
  audio file: preprocessed_non water sound sample  Start: 0 Stop: 21
    non_water
      0.0 0.5
      0.5 1.0
      1.0 1.5
      1.5 2.0
      2.0 2.5
      2.5 3.0
      3.0 3.5
      3.5 4.0
      4.0 4.5
      4.5 5.0
      5.0 5.5
      5.5 6.0
      6.0 6.5
      6.5 7.0
      7.0 7.5
      7.5 8.0
      8.0 8.5
      8.5 9.0
      9.0 9.5
      9.5 10.0
      10.0 10.5
      10.5 11.0
      11.0 11.5
      11.5 12.0
      12.0 12.5
      12.5 13.0
      13.0 13.5
      13.5 14.0
      14.0 14.5
      14.5 15.0
      15.0 15.5
      15.5 16.0
      16.0 16.5
      16.5 17.0
      17.0 17.5
      17.5 18.0
      18.0 18.5
      18.5 19.0
      19.0 19.5
      19.5 20.0
      20.0 20.5
      20.5 21
CSV file: time stamp 02.csv
  audio file: preprocessed_water sound sample  Start: 0 Stop: 27
    water
      0.0 0.5
      0.5 1.0
      1.0 1.5
      1.5 2.0
      2.0 2.5
      2.5 3.0
      3.0 3.5
      3.5 4.0
      4.0 4.5
      4.5 5.0
      5.0 5.5
 

In [101]:
# we got everything splitted

# Preprocessing Test Dataset

In [125]:
# Above is for sample dataset. for the test dataset, we just need to split them as per the SPLIT_LENGTH
# find a way to know how long is the audio file

from pydub import AudioSegment

# test_dataset_folder = r'test_test_dataset'
# split_test_dataset_folder = r'test_split_test_dataset'

test_dataset_folder = r'test_1_home_recorded\test_test_dataset'
split_test_dataset_folder = r'test_1_home_recorded\test_split_test_dataset'

# audio_file = "test_1.wav"
audio_file = "preprocessed_test data water and non_water.wav"
file_path = pathlib.Path(test_dataset_folder+"/"+audio_file)

audio = AudioSegment.from_file(file_path)
print(audio.duration_seconds)

#OK got the way to know how long the audio file is

35.5033125


In [126]:
start_split_time = np.arange(0.0, audio.duration_seconds, SPLIT_LENGTH).tolist()
stop_split_time = start_split_time[1:]+[audio.duration_seconds]

list(zip(start_split_time, stop_split_time))

#OK, got the start and stop split time

[(0.0, 0.5),
 (0.5, 1.0),
 (1.0, 1.5),
 (1.5, 2.0),
 (2.0, 2.5),
 (2.5, 3.0),
 (3.0, 3.5),
 (3.5, 4.0),
 (4.0, 4.5),
 (4.5, 5.0),
 (5.0, 5.5),
 (5.5, 6.0),
 (6.0, 6.5),
 (6.5, 7.0),
 (7.0, 7.5),
 (7.5, 8.0),
 (8.0, 8.5),
 (8.5, 9.0),
 (9.0, 9.5),
 (9.5, 10.0),
 (10.0, 10.5),
 (10.5, 11.0),
 (11.0, 11.5),
 (11.5, 12.0),
 (12.0, 12.5),
 (12.5, 13.0),
 (13.0, 13.5),
 (13.5, 14.0),
 (14.0, 14.5),
 (14.5, 15.0),
 (15.0, 15.5),
 (15.5, 16.0),
 (16.0, 16.5),
 (16.5, 17.0),
 (17.0, 17.5),
 (17.5, 18.0),
 (18.0, 18.5),
 (18.5, 19.0),
 (19.0, 19.5),
 (19.5, 20.0),
 (20.0, 20.5),
 (20.5, 21.0),
 (21.0, 21.5),
 (21.5, 22.0),
 (22.0, 22.5),
 (22.5, 23.0),
 (23.0, 23.5),
 (23.5, 24.0),
 (24.0, 24.5),
 (24.5, 25.0),
 (25.0, 25.5),
 (25.5, 26.0),
 (26.0, 26.5),
 (26.5, 27.0),
 (27.0, 27.5),
 (27.5, 28.0),
 (28.0, 28.5),
 (28.5, 29.0),
 (29.0, 29.5),
 (29.5, 30.0),
 (30.0, 30.5),
 (30.5, 31.0),
 (31.0, 31.5),
 (31.5, 32.0),
 (32.0, 32.5),
 (32.5, 33.0),
 (33.0, 33.5),
 (33.5, 34.0),
 (34.0, 34.5),
 (34

In [127]:
for idx, elem in enumerate(list(zip(start_split_time, stop_split_time))):
    print(f"index: {idx} start: {elem[0]} stop: {elem[1]}")

#ok it works.

index: 0 start: 0.0 stop: 0.5
index: 1 start: 0.5 stop: 1.0
index: 2 start: 1.0 stop: 1.5
index: 3 start: 1.5 stop: 2.0
index: 4 start: 2.0 stop: 2.5
index: 5 start: 2.5 stop: 3.0
index: 6 start: 3.0 stop: 3.5
index: 7 start: 3.5 stop: 4.0
index: 8 start: 4.0 stop: 4.5
index: 9 start: 4.5 stop: 5.0
index: 10 start: 5.0 stop: 5.5
index: 11 start: 5.5 stop: 6.0
index: 12 start: 6.0 stop: 6.5
index: 13 start: 6.5 stop: 7.0
index: 14 start: 7.0 stop: 7.5
index: 15 start: 7.5 stop: 8.0
index: 16 start: 8.0 stop: 8.5
index: 17 start: 8.5 stop: 9.0
index: 18 start: 9.0 stop: 9.5
index: 19 start: 9.5 stop: 10.0
index: 20 start: 10.0 stop: 10.5
index: 21 start: 10.5 stop: 11.0
index: 22 start: 11.0 stop: 11.5
index: 23 start: 11.5 stop: 12.0
index: 24 start: 12.0 stop: 12.5
index: 25 start: 12.5 stop: 13.0
index: 26 start: 13.0 stop: 13.5
index: 27 start: 13.5 stop: 14.0
index: 28 start: 14.0 stop: 14.5
index: 29 start: 14.5 stop: 15.0
index: 30 start: 15.0 stop: 15.5
index: 31 start: 15.5 stop

In [128]:
# now let's get the all of the file name 

test_dataset_dir = pathlib.Path(test_dataset_folder)
dir_list = os.listdir(test_dataset_dir)
dir_list

['preprocessed_test data water and non_water.wav']

In [133]:
#now, let's combine it all

test_dataset_dict = {'file': [], 'split': [], 'start': [], 'stop': []}

for audio_file in dir_list:
    file_path = pathlib.Path(test_dataset_folder+"/"+audio_file)
    audio = AudioSegment.from_file(file_path)
    print(f"audio file = {audio_file}")
    start_split_time = np.arange(0.0, audio.duration_seconds, SPLIT_LENGTH).tolist()
    stop_split_time = start_split_time[1:]+[audio.duration_seconds]
    for idx, elem in enumerate(list(zip(start_split_time, stop_split_time))):
        output_file_name = audio_file[:-4] + "_" + str(idx) + ".wav"
        print(f"  index: {idx} start: {elem[0]} stop: {elem[1]} output_file_name: {output_file_name}")
        split_audio(test_dataset_folder+'\\'+audio_file, elem[0], elem[1], split_test_dataset_folder+"\\"+output_file_name)
        test_dataset_dict['file'].append(audio_file[:-4]) 
        test_dataset_dict['split'].append(output_file_name[:-4]) 
        test_dataset_dict['start'].append(elem[0]) 
        test_dataset_dict['stop'].append(elem[1]) 

# print(test_dataset_dict)

audio file = preprocessed_test data water and non_water.wav
  index: 0 start: 0.0 stop: 0.5 output_file_name: preprocessed_test data water and non_water_0.wav
  index: 1 start: 0.5 stop: 1.0 output_file_name: preprocessed_test data water and non_water_1.wav
  index: 2 start: 1.0 stop: 1.5 output_file_name: preprocessed_test data water and non_water_2.wav
  index: 3 start: 1.5 stop: 2.0 output_file_name: preprocessed_test data water and non_water_3.wav
  index: 4 start: 2.0 stop: 2.5 output_file_name: preprocessed_test data water and non_water_4.wav
  index: 5 start: 2.5 stop: 3.0 output_file_name: preprocessed_test data water and non_water_5.wav
  index: 6 start: 3.0 stop: 3.5 output_file_name: preprocessed_test data water and non_water_6.wav
  index: 7 start: 3.5 stop: 4.0 output_file_name: preprocessed_test data water and non_water_7.wav
  index: 8 start: 4.0 stop: 4.5 output_file_name: preprocessed_test data water and non_water_8.wav
  index: 9 start: 4.5 stop: 5.0 output_file_name:

In [134]:
test_dataset_df = pd.DataFrame.from_dict(test_dataset_dict)
test_dataset_df

,file,split,start,stop
0,preprocessed_test data water and non_water,preprocessed_test data water and non_water_0,0.0,0.500000
1,preprocessed_test data water and non_water,preprocessed_test data water and non_water_1,0.5,1.000000
2,preprocessed_test data water and non_water,preprocessed_test data water and non_water_2,1.0,1.500000
3,preprocessed_test data water and non_water,preprocessed_test data water and non_water_3,1.5,2.000000
4,preprocessed_test data water and non_water,preprocessed_test data water and non_water_4,2.0,2.500000
...,...,...,...,...
67,preprocessed_test data water and non_water,preprocessed_test data water and non_water_67,33.5,34.000000
68,preprocessed_test data water and non_water,preprocessed_test data water and non_water_68,34.0,34.500000
69,preprocessed_test data water and non_water,preprocessed_test data water and non_water_69,34.5,35.000000
70,preprocessed_test data water and non_water,preprocessed_test data water and non_water_70,35.0,35.500000


In [135]:
test_dataset_df.to_csv("test_dataset_df.csv")

In [102]:
# we got everything splitted